In [1]:
from types import SimpleNamespace 
from datetime import datetime, timedelta
import akshare as ak
import pandas as pd
import multiprocessing
# import threading
import threading
import queue
import psutil
import time
import talib
from concurrent.futures  import ThreadPoolExecutor, ProcessPoolExecutor, as_completed
import numpy as np
import kline_daily
import requests
# import cloudscraper
import logging

In [2]:
logging.basicConfig( 
    filename='all.log',           # 日志文件名 
    filemode='a',                # 模式：'a'追加/'w'覆盖 
    level=logging.DEBUG,         # 记录DEBUG及以上级别 
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',  # 日志格式 
    datefmt='%Y-%m-%d %H:%M:%S'  # 时间格式 
)
 
# 创建日志记录器 
logger = logging.getLogger('error_logger') 
logger.setLevel(logging.ERROR)   # 仅处理 ERROR 及以上 
 
# 创建文件处理器（仅写 ERROR）
file_handler = logging.FileHandler('error.log') 
file_handler.setLevel(logging.ERROR) 
 
# 设置日志格式 
formatter = logging.Formatter('%(asctime)s - %(message)s')
file_handler.setFormatter(formatter) 
 
# 添加到记录器 
logger.addHandler(file_handler) 
 

In [5]:
response = requests.get('https://api.proxyscrape.com/v4/free-proxy-list/get?request=display_proxies&proxy_format=protocolipport&format=text',  timeout=30)

In [6]:
print(response.status_code)

200


In [22]:
proxies = response.text.replace("socks4",  "https")
proxies_ls = [proxy for proxy in proxies.split('\r\n')  if proxy != ""]
proxies_map = []
for url in proxies_ls:
    try:
        # 按 "://" 分割协议和地址（最多分割1次）
        protocol, _address = url.split('://',  1)
        if protocol == "http":
            proxies_map.append({protocol: url})
    except ValueError:
        # 处理无效格式（如缺少 ://）
        print(f"跳过无效URL: {url}")
print(proxies_map)

[{'http': 'http://103.160.204.11:80'}, {'http': 'http://45.67.215.255:80'}, {'http': 'http://89.116.250.110:80'}, {'http': 'http://170.114.45.120:80'}, {'http': 'http://108.162.198.74:80'}, {'http': 'http://69.84.182.23:80'}, {'http': 'http://141.101.120.172:80'}, {'http': 'http://102.177.176.29:80'}, {'http': 'http://89.116.250.164:80'}, {'http': 'http://69.84.182.242:80'}, {'http': 'http://108.162.198.81:80'}, {'http': 'http://89.116.250.16:80'}, {'http': 'http://45.67.215.55:80'}, {'http': 'http://45.67.215.234:80'}, {'http': 'http://69.84.182.217:80'}, {'http': 'http://108.162.195.131:80'}, {'http': 'http://216.205.52.23:80'}, {'http': 'http://170.114.45.152:80'}, {'http': 'http://102.177.176.125:80'}, {'http': 'http://216.205.52.0:80'}, {'http': 'http://69.84.182.185:80'}, {'http': 'http://102.177.176.47:80'}, {'http': 'http://141.101.115.254:80'}, {'http': 'http://69.84.182.156:80'}, {'http': 'http://89.116.250.250:80'}, {'http': 'http://89.116.250.218:80'}, {'http': 'http://185.

In [23]:
print(proxies_map)

[{'http': 'http://103.160.204.11:80'}, {'http': 'http://45.67.215.255:80'}, {'http': 'http://89.116.250.110:80'}, {'http': 'http://170.114.45.120:80'}, {'http': 'http://108.162.198.74:80'}, {'http': 'http://69.84.182.23:80'}, {'http': 'http://141.101.120.172:80'}, {'http': 'http://102.177.176.29:80'}, {'http': 'http://89.116.250.164:80'}, {'http': 'http://69.84.182.242:80'}, {'http': 'http://108.162.198.81:80'}, {'http': 'http://89.116.250.16:80'}, {'http': 'http://45.67.215.55:80'}, {'http': 'http://45.67.215.234:80'}, {'http': 'http://69.84.182.217:80'}, {'http': 'http://108.162.195.131:80'}, {'http': 'http://216.205.52.23:80'}, {'http': 'http://170.114.45.152:80'}, {'http': 'http://102.177.176.125:80'}, {'http': 'http://216.205.52.0:80'}, {'http': 'http://69.84.182.185:80'}, {'http': 'http://102.177.176.47:80'}, {'http': 'http://141.101.115.254:80'}, {'http': 'http://69.84.182.156:80'}, {'http': 'http://89.116.250.250:80'}, {'http': 'http://89.116.250.218:80'}, {'http': 'http://185.

In [9]:
# total_len = len(df)
physical_cpus = psutil.cpu_count(logical=False)
logical_cpus = psutil.cpu_count(logical=True)

log_phy_ratio = int(logical_cpus/physical_cpus)
# phy_cpu_length = int(total_len/physical_cpus)
args = SimpleNamespace(
    datedelta = 50,
    start_epochs = 8
    )
today = datetime.now()
start_day = today - timedelta(days=args.datedelta)
test_day = today - timedelta(days=1)

# 格式化日期为YYYYMMDD格式
formatted_today = today.strftime('%Y%m%d')
formatted_test_day = test_day.strftime('%Y%m%d')
formatted_start_day = start_day.strftime('%Y%m%d')

In [10]:
def validate_proxy(proxies, result_queue):
    """验证代理有效性 [6]()"""
    test_url = "http://icanhazip.com"   # 测试网站 
    for proxy in proxies:
        # print("Testing proxy:", proxy)
        try:
            start_time = time.time() 
            resp = requests.get(test_url,  proxies=proxy, timeout=5)
            latency = int((time.time()  - start_time))  # 计算延迟 
            # print(resp.status_code, type(resp.status_code),resp.text,proxy)
            if resp.status_code  == 200:
                result_queue.put((latency, proxy))
        except Exception:
            pass

In [24]:
aspls = np.array_split(proxies_map, logical_cpus)
# 然后每个子数组是numpy数组，可以转成列表
chunked_list = [arr.tolist() for arr in aspls]

validate_proxies_queue = queue.Queue()
validate_proxies_ls = []

process_lock = multiprocessing.Lock()

with ThreadPoolExecutor(max_workers=logical_cpus) as executor:
    futures = [executor.submit(validate_proxy, proxies, validate_proxies_queue)
        for proxies in chunked_list]
    for future in as_completed(futures):
        try:
            future.result()
        except Exception as e:
            print("Error in got results thread:", e)

while not validate_proxies_queue.empty():
    validate_proxies_ls.append(validate_proxies_queue.get())

validate_proxies_ls =sorted(validate_proxies_ls, key=lambda x: x[0])

In [25]:
for i in validate_proxies_ls:
    validate_proxies_queue.put(i[1])

In [26]:
print(validate_proxies_queue.empty())

False


In [56]:
user_agent = 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.104 Safari/537.36 Core/1.53.4295.400'
url = "https://push2his.eastmoney.com/api/qt/stock/kline/get"
params = {
    "fields1": "f1,f2,f3,f4,f5,f6",
    "fields2": "f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f116",
    "ut": "fa5fd1943c7b386f172d6893dbfba10b",
    "klt": "101",  # 日线
    "fqt": "1",  # 前复权
    "secid": f"{0}.{"000001"}",
    "beg": formatted_start_day,
    "end": formatted_today,
}
headers = {'User-Agent': user_agent}
rs= requests.get(url, headers=headers, params=params, timeout=5, proxies={"http":validate_proxies_ls[2][1]["http"]})
print(rs)

ProxyError: HTTPSConnectionPool(host='push2his.eastmoney.com', port=443): Max retries exceeded with url: /api/qt/stock/kline/get?fields1=f1%2Cf2%2Cf3%2Cf4%2Cf5%2Cf6&fields2=f51%2Cf52%2Cf53%2Cf54%2Cf55%2Cf56%2Cf57%2Cf58%2Cf59%2Cf60%2Cf61%2Cf116&ut=fa5fd1943c7b386f172d6893dbfba10b&klt=101&fqt=1&secid=0.000001&beg=20250719&end=20250907 (Caused by ProxyError('Unable to connect to proxy', RemoteDisconnected('Remote end closed connection without response')))

In [41]:
print(len(validate_proxies_ls))

1588


In [37]:
with open('today_suggestions.txt', 'w', encoding='utf-8') as today_suggestions,open('history_suggestions.txt', 'a', encoding='utf-8') as history_suggestions:
    today_suggestions.write(formatted_today + "!!!!!!!!~~~~~~~~~~~~~~\n")
    history_suggestions.write(formatted_today + "!!!!!!!!!!!~~~~~~~~~~~~\n")
# df = pd.read_csv('data.csv',  dtype={0: str})
df = pd.read_csv('mainboard_stocks.csv', dtype={'code': str})

In [38]:
stock_codes = list(set(df['code']))
print(len(df) == len(stock_codes))
df.head()

True


,code,name
0,000001,平安银行
1,000002,万 科Ａ
2,000004,*ST国华
3,000006,深振业Ａ
4,000007,全新好


In [46]:
def stock_data_getter(stock_codes, formatted_start_day, formatted_today, validate_proxies_queue):
    res_ls = []
    if validate_proxies_queue.empty():
        return []
    proxy = validate_proxies_queue.get()
    i = 0
    while i < len(stock_codes):
        time.sleep(0.8) #avoid abandon from remote
        stock_code = stock_codes[i]
        i += 1
        stock_zh_a_hist_df = kline_daily.stock_zh_a_hist_with_proxy(symbol=stock_code, start_date=formatted_start_day, end_date=formatted_today, proxy={"http":proxy["http"]})
        if stock_zh_a_hist_df is None:
            if validate_proxies_queue.empty():
                print("proxy ran out")
                return []
            proxy = validate_proxies_queue.get()
            i -= 1
        elif stock_zh_a_hist_df.empty:
            print("wrong code:",stock_code)
        else:
            res_ls.append((stock_code, stock_zh_a_hist_df))
    print("stock code:",stock_codes[-1],"data collection finished:", time.time())
    return res_ls

In [47]:
aspls = np.array_split(stock_codes, logical_cpus)
# 然后每个子数组是numpy数组，可以转成列表
chunked_list = [arr.tolist() for arr in aspls]

process_lock = multiprocessing.Lock()

stock_data = []

with ThreadPoolExecutor(max_workers=logical_cpus) as executor:
    futures = [executor.submit(stock_data_getter, stock_codes_ls, formatted_start_day, formatted_today, validate_proxies_queue)
        for stock_codes_ls in chunked_list]
    for future in as_completed(futures):
        try:
            result = future.result() 
            stock_data += result
        except Exception as e:
            print("Error in got results thread:", e)

proxy ran out
proxy ran out
proxy ran out
proxy ran out
proxy ran out
proxy ran out
proxy ran out
proxy ran out
proxy ran out
proxy ran out
proxy ran out
proxy ran out
proxy ran out
proxy ran out
proxy ran out
proxy ran out
proxy ran out
proxy ran out
proxy ran out
proxy ran out
proxy ran out
proxy ran out
proxy ran out
proxy ran out
proxy ran out
proxy ran out
proxy ran out
proxy ran out
proxy ran out
proxy ran out
proxy ran out
proxy ran out


In [54]:
print(len(stock_data),len(stock_codes))

0 3032


In [ ]:
sdaspls = np.array_split(stock_data, physical_cpus)
sd_chunked_list = [arr.tolist() for arr in sdaspls]

In [ ]:
def check_parameter(stock_codes_ls):
    for stock_code, stock_zh_df in stock_codes_ls:
        open = stock_zh_df["开盘"]
        close = stock_zh_df["收盘"]
        high = stock_zh_df["最高"]
        low = stock_zh_df["最低"]
        volume = stock_zh_df["成交量"]
        turnover = stock_zh_df["换手率"]

        ma5 = talib.SMA(close[-5:], timeperiod=5)
        ma10 = talib.SMA(close[-10:], timeperiod=10)
        vol_ma5 = talib.SMA(volume[-5:], timeperiod=5)

        macd, macdsignal, macdhist = talib.MACDEXT(close, fastperiod=12, fastmatype=0, slowperiod=20, slowmatype=0, signalperiod=9, signalmatype=0)

        rsi = talib.RSI(close, timeperiod=14)  # RSI相对强弱指标
        cci = talib.CCI(high, low, close, timeperiod=20)  # 顺势指标
        # money_flow = (2*close - low - high) / (high - low) * volume  # 简易资金流
        money_flow = talib.MFI(high, low, close, volume, timeperiod=9)
        money_flow_max= money_flow.shift(3).rolling(4).max()
        turnover_mean= money_flow.shift(1).rolling(5).mean()
        today_candlestick = (close.iloc[-3] - open.iloc[-3])
        yesterday_candlestick = (close.iloc[-4] - open.iloc[-4])

        if (close.iloc[-3] > ma5.iloc[-3] and ma5.iloc[3] > ma10.iloc[-3]) and \
                volume.iloc[-3] > vol_ma5.iloc[-3] * 3.8 and turnover.iloc[-1] > turnover_mean * 1.5 and \
                (macd.iloc[-3] > macdsignal.iloc[-3] and macd.iloc[-4] < macdsignal.iloc[-4] and macdhist.iloc[-3] > abs(macdhist.iloc[-4])):
            if close.iloc[-1] > close.iloc[-2]:
                print(f"""stock_code: {stock_code} result: 趋势启动+量价齐升型 \n """)
        elif (rsi.iloc[-3] <30) and (cci.iloc[-3] < -150) and \
            today_candlestick > 0.5 * yesterday_candlestick and turnover.iloc[-1] > turnover_mean * 1.2 and \
            money_flow.iloc[-3] > money_flow_max:
            if close.iloc[-1] > close.iloc[-2]:
                print(f"""stock_code: {stock_code} result: 超跌反转+资金异动型 \n """)


In [ ]:
# with ProcessPoolExecutor(max_workers=physical_cpus) as executor:
#     futures = [executor.submit(check_parameter, sd_ls)
#         for sd_ls in sd_chunked_list]
#     for future in as_completed(futures):
#         try:
#             future.result() 
#         except Exception as e:
#             print("Error in process:", e)

In [ ]:
def run_analysis(stock_codes_ls, process_lock):
    res_ls = []
    for stock_code, stock_zh_df in stock_codes_ls:
        open = stock_zh_df["开盘"]
        close = stock_zh_df["收盘"]
        high = stock_zh_df["最高"]
        low = stock_zh_df["最低"]
        volume = stock_zh_df["成交量"]
        turnover = stock_zh_df["换手率"]

        ma5 = talib.SMA(close[-5:], timeperiod=5)
        ma10 = talib.SMA(close[-10:], timeperiod=10)
        vol_ma5 = talib.SMA(volume[-5:], timeperiod=5)

        macd, macdsignal, macdhist = talib.MACDEXT(close, fastperiod=12, fastmatype=0, slowperiod=20, slowmatype=0, signalperiod=9, signalmatype=0)

        rsi = talib.RSI(close, timeperiod=14)  # RSI相对强弱指标
        cci = talib.CCI(high, low, close, timeperiod=20)  # 顺势指标
        # money_flow = (2*close - low - high) / (high - low) * volume  # 简易资金流
        money_flow = talib.MFI(high, low, close, volume, timeperiod=9)
        money_flow_max= money_flow.shift(1).rolling(4).max()
        turnover_mean= money_flow.shift(1).rolling(5).mean()
        today_candlestick = abs(close.iloc[-1] - open.iloc[-1])
        yesterday_candlestick = abs(close.iloc[-2] - open.iloc[-2])

        if (close.iloc[-1] > ma5.iloc[-1] and ma5.iloc[1] > ma10.iloc[-1]) and \
                volume.iloc[-1] > vol_ma5.iloc[-1] * 1.8 and turnover.iloc[-1] > turnover_mean * 1.5 and \
                (macd.iloc[-1] > macdsignal.iloc[-1] and macd.iloc[-2] < macdsignal.iloc[-2] and macdhist.iloc[-1] > abs(macdhist.iloc[-2])):
            res_ls.append(f"""stock_code: {stock_code} result: 趋势启动+量价齐升型 \n """)
        elif (rsi.iloc[-1] <30) and (cci.iloc[-1] < -150) and \
            today_candlestick > 0.5 * yesterday_candlestick and turnover.iloc[-1] > turnover_mean * 1.2 and \
            money_flow.iloc[-1] > money_flow_max:
            res_ls.append(f"""stock_code: {stock_code} result: 超跌反转+资金异动型 \n """)

    with process_lock:
        with open('today_suggestions.txt', 'a', encoding='utf-8') as today_suggestions,open('history_suggestions.txt', 'a', encoding='utf-8') as history_suggestions:
            for result in res_ls:
                today_suggestions.write(result)
                history_suggestions.write(result)
    return  

In [ ]:
# with ProcessPoolExecutor(max_workers=physical_cpus) as executor:
#     futures = [executor.submit(run_analysis, args=(stock_codes_ls, process_lock))
#         for stock_codes_ls in chunked_list]
#     for future in as_completed(futures):
#         try:
#             future.result() 
#         except Exception as e:
#             print("Error in thread:", e)


In [ ]:
# collect_and_analyze_data(["000001","000002"], log_phy_ratio, formatted_start_day, formatted_today, lock, run_analysis)

In [53]:
# sd = ak.stock_zh_a_hist(symbol="000001", period="daily", start_date=formatted_start_day, end_date=formatted_today, adjust="")
# print(sd)
# df_shanghai = ak.index_zh_a_hist( 
#     symbol="000001",      # 上证指数代码（固定为000001）
#     period="daily",       # 数据周期：daily（日线）、weekly（周线）、monthly（月线）
#     start_date="20200101", # 起始日期（格式：YYYYMMDD）
#     end_date="20250904",   # 结束日期（默认为当前日期）
# )

# # 查看前5行数据 
# print(df_shanghai.head()) 

In [ ]:
# def get_mainboard_stocks_ak():
#     """使用akshare获取主板股票列表"""
#     # 获取所有上市公司的基本信息
#     stock_info = ak.stock_info_a_code_name()
    
#     # 筛选主板股票
#     mainboard_stocks = stock_info[stock_info['code'].str.startswith(('600', '601', '603', '605', '000', '002'))]
    
#     return mainboard_stocks

# # 获取主板股票
# mainboard_stocks_ak = get_mainboard_stocks_ak()
# print(mainboard_stocks_ak.head())
# mainboard_stocks_ak.to_csv('mainboard_stocks.csv', index=False, encoding='utf-8-sig')

In [ ]:
# # date="20200331"; choice of {"XXXX0331", "XXXX0630", "XXXX0930", "XXXX1231"}; 从 20081231 开始
# stock_yjyg_em_df = ak.stock_yjyg_em(date="20250630")
# stock_yjyg_em_df_sorted_desc = stock_yjyg_em_df.sort_values(by=stock_yjyg_em_df.columns[6], ascending=False) #'业绩变动幅度'
# # print(stock_yjyg_em_df_sorted_desc.head(10))
# print(stock_yjyg_em_df_sorted_desc.iloc[:, [1,6]].head(10))

In [ ]:
# class ProxyManager:
#     """代理IP管理器"""

#     def __init__(self, proxy_api_url, max_retry=3):
#         self.proxy_api_url = proxy_api_url
#         self.max_retry = max_retry

#     def get_valid_proxy(self):
#         """获取有效的代理IP"""
#         for attempt in range(self.max_retry):
#             try:
#                 resp = requests.get(self.proxy_api_url, timeout=5)
#                 proxy_json = resp.json()
#                 proxy_data = proxy_json["data"][0]
#                 server = proxy_data["server"]
#                 ip, port = server.split(":")
#                 proxy = {"http": f"http://{ip}:{port}", "https": f"http://{ip}:{port}"}

#                 # 验证代理可用性
#                 test_url = "http://quote.eastmoney.com"
#                 test = requests.get(test_url, proxies=proxy, timeout=5)
#                 if test.status_code == 200:
#                     print(f"代理可用: {ip}:{port}")
#                     return proxy
#             except Exception as e:
#                 print(f"获取代理失败，第{attempt + 1}次尝试: {e}")

#         print(f"未能获取有效代理")
#         return None